In [58]:
from keras.preprocessing import image
from matplotlib import pyplot as plt

In [1]:
! mkdir ~/.kaggle

In [5]:
! cp kaggle.json ~/.kaggle/

In [6]:
! chmod 600 ~/.kaggle kaggle.json

In [7]:
! kaggle datasets download -d eabdul/dogs-cats-horses-humans-dataset

Dataset URL: https://www.kaggle.com/datasets/eabdul/dogs-cats-horses-humans-dataset
License(s): unknown
 99% 38.0M/38.6M [00:03<00:00, 22.8MB/s]
100% 38.6M/38.6M [00:03<00:00, 12.2MB/s]


In [8]:
! unzip dogs-cats-horses-humans-dataset.zip

Archive:  dogs-cats-horses-humans-dataset.zip
  inflating: dataset/cats/cat.1.jpg  
  inflating: dataset/cats/cat.10.jpg  
  inflating: dataset/cats/cat.100.jpg  
  inflating: dataset/cats/cat.101.jpg  
  inflating: dataset/cats/cat.102.jpg  
  inflating: dataset/cats/cat.103.jpg  
  inflating: dataset/cats/cat.104.jpg  
  inflating: dataset/cats/cat.105.jpg  
  inflating: dataset/cats/cat.106.jpg  
  inflating: dataset/cats/cat.107.jpg  
  inflating: dataset/cats/cat.108.jpg  
  inflating: dataset/cats/cat.109.jpg  
  inflating: dataset/cats/cat.11.jpg  
  inflating: dataset/cats/cat.110.jpg  
  inflating: dataset/cats/cat.111.jpg  
  inflating: dataset/cats/cat.112.jpg  
  inflating: dataset/cats/cat.113.jpg  
  inflating: dataset/cats/cat.114.jpg  
  inflating: dataset/cats/cat.115.jpg  
  inflating: dataset/cats/cat.116.jpg  
  inflating: dataset/cats/cat.117.jpg  
  inflating: dataset/cats/cat.118.jpg  
  inflating: dataset/cats/cat.119.jpg  
  inflating: dataset/cats/cat.12.jpg  

In [9]:
! ls

dataset  dogs-cats-horses-humans-dataset.zip  kaggle.json  sample_data


In [48]:
import os
os.listdir("dataset/dataset")

['dogs', 'humans', 'cats', 'horses']

In [49]:
folders = os.listdir("dataset/dataset")
print(folders)

['dogs', 'humans', 'cats', 'horses']


In [50]:
image_data = []
labels = []

label_dict = {"cats": 0,
              "dogs": 1,
              "horses" : 2,
              "humans": 3}

In [51]:
for ix in folders:
  path = os.path.join("dataset/dataset",ix)
  print(path)

dataset/dataset/dogs
dataset/dataset/humans
dataset/dataset/cats
dataset/dataset/horses


In [52]:
for ix in folders:
  path = os.path.join("dataset/dataset",ix)
  for im in os.listdir(path):
    img = image.load_img(os.path.join(path,im),target_size=(224,224))
    img_array = image.img_to_array(img)
    image_data.append(img_array) # creating a list
    labels.append(label_dict[ix]) #ix is the folder name, to show
    # which label theyb belong out of 4

In [56]:
print(len(image_data))
print(len(labels))

808
808


In [55]:
# comibing images  with the labels into a zipped list !! why ?
combined = list (zip(image_data,labels))
# shuffle those pairs
import random
random.shuffle(combined)

In [57]:
image_data[:], labels[:] = zip (*combined) # what code this d

This line of code unpacks and assigns shuffled image and label data to separate variables. Here's a breakdown:

1. **Understanding `combined`:** Before this line, `combined` is likely a list of paired items (tuples), where each tuple contains an image and its corresponding label. For instance, `combined` might look like:
   ```python
   combined = [(image1, label1), (image2, label2), ...]
   ```

2. **Using `zip(*combined)`:** This operation effectively "unzips" the list of tuples into two separate lists: one for images and one for labels. The `*` operator unpacks `combined` so `zip` can group the first elements together (all images) and the second elements together (all labels). The result is something like:
   ```python
   images = [image1, image2, ...]
   labels = [label1, label2, ...]
   ```

3. **Assignment to `image_data[:]` and `labels[:]`:** The `image_data[:]` and `labels[:]` notation indicates that the contents of these lists are being modified in place (instead of creating new lists). This is useful if `image_data` and `labels` are already defined lists, and you want to update them directly with the new values.

### Summary
The line:
```python
image_data[:], labels[:] = zip(*combined)
```
splits the `combined` list of tuples into separate lists of images and labels, then directly updates the `image_data` and `labels` lists with these values. This is often used when shuffling paired data (e.g., images and labels) to keep the pairs aligned after shuffling.

In [59]:
print(labels[:5])

[1, 2, 1, 2, 3]


In [67]:
import numpy as np
from tensorflow.keras.utils import to_categorical

In [62]:
X_train = np.array(image_data)
y_train = np.array(labels)

In [63]:
print (X_train.shape)
print (y_train.shape)

(808, 224, 224, 3)
(808,)


In [64]:
# WE are going to use MLP in the end so , Y should be one hot encoded!!

In [69]:
y_train = to_categorical(y_train,4)

In [70]:
print (y_train.shape)

(808, 4)


In [71]:
# create the model REsnet 50 model

In [74]:
from keras.applications.resnet50 import ResNet50
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import *
from matplotlib import pyplot as plt

In [77]:
model = ResNet50(include_top=False,weights ="imagenet",input_shape=(224,224,3))

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [78]:
model.summary()

Model: "resnet50"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 230, 230, 3)    │              0 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 112, 112, 64)   │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 112, 112, 64)   │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 112, 112, 64)   │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 114, 114, 64)   │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 56, 56, 64)     │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 56, 56, 64)     │          4,160 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 56, 56, 64)     │         36,928 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, 56, 56, 256)    │         16,640 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_conv       │ (None, 56, 56, 256)    │         16,640 │ conv2_block1_2_relu[0… │
│ (Conv2D)             

 Total params: 23,587,712 (89.98 MB)

 Trainable params: 23,534,592 (89.78 MB)

 Non-trainable params: 53,120 (207.50 KB)